# Check `open_data`
* https://github.com/cal-itp/data-analyses/tree/more-open-data-refactor/open_data
* download_warehouse_inputs (this is what to run to download for the month)
* create_routes_data2 
* create_stops_data2
* i got rid of on_shn as dummy, instead, it's just put into shn_route, where if it's not, it says not_50ft_from_shn, but if it is, it says 5, 10, whatever other freeways

In [2]:
from functools import cache
import geopandas as gpd
import pandas as pd
import google.auth

from calitp_data_analysis import geography_utils, utils
from calitp_data_analysis.gcs_pandas import GCSPandas
from update_vars import GTFS_DATA_DICT, file_name
credentials, project = google.auth.default()

/opt/conda/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
@cache
def gcs_pandas():
    return GCSPandas()


## `create_stops_data2`

In [4]:
OPEN_DATA_GCS = "gs://calitp-analytics-data/data-analyses/open_data/"

In [5]:
analysis_month = "2025-12-01"

In [7]:
f"export/ca_transit_stops_{analysis_month}"

'export/ca_transit_stops_2025-12-01'

In [11]:
stops2_gdf = gpd.read_parquet(f"{OPEN_DATA_GCS}export/ca_transit_stops_{analysis_month}.parquet",
                             storage_options={"token": credentials.token})

In [12]:
stops2_gdf.drop(columns = ["geometry"]).sample(3)

,name,stop_id,stop_name,total_stop_arrivals,routetypes,route_ids_served,n_hours_in_service,n_arrivals,n_routes,meters_to_ca_state_highway,analysis_name,district_name
56802,LA DOT Schedule,298882,6th & Burnside (Eastbound),774,[3],[563],14.0,25,1,2916.1,City of Los Angeles,07 - Los Angeles / Ventura
42241,StanRTA Schedule,1270,A St & Old Stockton Rd,433,[3],[60],17.0,15,1,127.5,Stanislaus Regional Transit Authority,10 - Stockton
29799,Bay Area 511 Muni Schedule,14269,Geary Blvd & 25th Ave,15762,[3],"[38R, 38]",24.0,508,2,1013.8,City and County of San Francisco,04 - Bay Area / Oakland


## `create_routes_data2`

In [13]:
create_routes_data2_gdf = gpd.read_parquet(f"{OPEN_DATA_GCS}export/ca_transit_routes_{analysis_month}.parquet",
                             storage_options={"token": credentials.token})

In [14]:
create_routes_data2_gdf.drop(columns = ["geometry"]).sample(3)

,name,shape_id,route_name,n_trips,route_type,route_length_feet,route_id,pct_route_on_hwy,shn_route,analysis_name,district_name
2062,Bay Area 511 Marin Schedule,53,654 Olive - Novato Loop,5,3,59000.63,654,0.7,101,Marin County Transit District,04 - Bay Area / Oakland
3685,SBMTD Schedule,shp-12X-53,12X Goleta Express,210,3,55948.39,12X,61.3,"101, 154, 217",Santa Barbara Metropolitan Transit District,05 - San Luis Obispo / Santa Barbara
3651,Riverside Schedule,shp-31-04,31 Hemet - Beaumont - Moreno Valley,80,3,199647.64,31,56.1,"10, 60, 79",Riverside Transit Agency,08 - San Bernardino / Riverside


In [16]:
create_routes_data2_gdf.loc[create_routes_data2_gdf.pct_route_on_hwy == 0].drop(columns = ["geometry"]).sample(3)

,name,shape_id,route_name,n_trips,route_type,route_length_feet,route_id,pct_route_on_hwy,shn_route,analysis_name,district_name
1290,BruinBus Schedule,a65182ca-d9a9-46e4-b6fd-30d61b552982,U901,43,3,25919.66,c4db64b5-a472-4c40-8772-5f5e7baab846,0.0,not_50ft_from_shn,"University of California, Los Angeles",07 - Los Angeles / Ventura
1296,LADPW Schedule,p_531879,Avocado Heights/Bassett/West Valinda Shuttle,20,3,25554.44,13050,0.0,not_50ft_from_shn,Los Angeles County,07 - Los Angeles / Ventura
62,Roseville Schedule,51924,C ROUTE C,14,3,31102.08,C,0.0,not_50ft_from_shn,City of Roseville,03 - Marysville / Sacramento


In [15]:
create_routes_data2_gdf.pct_route_on_hwy.describe()

count    4568.000000
mean       18.175131
std        27.707430
min         0.000000
25%         0.500000
50%         1.700000
75%        26.900000
max       100.000000
Name: pct_route_on_hwy, dtype: float64